In [2]:
#Import dependencies
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

In [3]:
#Predict sentiment using TextBlob sentiment analysis module
def predict_sentiment_textblob(text):
    value=TextBlob(text).sentiment.polarity
    return value

In [4]:
#Predict sentiment using Vader sentiment analysis module
def predict_sentiment_vader(text):
    analyser=SentimentIntensityAnalyzer()
    positive_value=analyser.polarity_scores(text)['pos']
    negative_value=analyser.polarity_scores(text)['neg']
    compound_value=analyser.polarity_scores(text)['compound']
    neutral_value=analyser.polarity_scores(text)['neu']
    return positive_value, negative_value, compound_value, neutral_value

In [5]:
#Word Pre-processing using NLTK modules
def custom_word_selection(text):
    word_tokens=word_tokenize(text)
    stop_words=set(stopwords.words("english"))
    filtered_words=[w for w in word_tokens if not w in stop_words]
#     ps = PorterStemmer()
#     stemmed_words=[ps.stem(w) for w in filtered_words]
#     total_words=[]
#     for i in stemmed_words:
#         total_words.append(i)
#         for j in filtered_words:
#             if j!=i:
#                 total_words.append(j)
#             else:
#                 continue
    pos_tagging=nltk.pos_tag(filtered_words)
    return pos_tagging

In [6]:
#Select import feature words such as Noun,Adjective,Adverb & Verb
#     JJ-Adjective
#     NN-Noun
#     RB-Adverb
#     VB-Verb
#     Dictionary contains almost 4.2k words
def custom_feature_selection(allword):
    relevant_pos_words=[]
    for i in pos_tagging:
        if (i[1].startswith('NN')==True or i[1].startswith('JJ')==True or i[1].startswith('RB')==True or i[1].startswith('VB')==True):
            relevant_pos_words.append(i)
        else:
            continue
    return relevant_pos_words

In [7]:
#Match the words with the master finanical dictionary 
def match_words_master(dictionary,word_list):
    matched_words=[]
    for i in word_list:
        for j in dictionary['Word']:
            if(i[0].upper()==j):
                matched_words.append(i[0])
            else:
                continue
    return matched_words

In [36]:
#Normalize the score of each word predicted by Vader Sentiment Analysis module
def normalize_scores(word_list):
    positive_score=negative_score=compound_score=neutral_score=0
    while(len(word_list)!=0):
        for i in word_list:
            positive_score=predict_sentiment_vader(i)[0]+positive_score
            negative_score=predict_sentiment_vader(i)[1]+negative_score
            compound_score=predict_sentiment_vader(i)[2]+compound_score
            neutral_score=predict_sentiment_vader(i)[3]+neutral_score
        return positive_score/len(word_list),negative_score/len(word_list),compound_score/len(word_list),neutral_score/len(word_list)

In [9]:
#Normalize the score of each word predicted by TextBlob Sentiment Analysis module
def normalize_scores1(word_list):
    overall_score=0
    for i in word_list:
        overall_score=TextBlob(i).sentiment.polarity+overall_score
    return overall_score/len(word_list)

In [10]:
#Load the sentimental dictionaries in pandas dataframe
dictionary_positive=pd.read_csv("sent_positive.csv",header=None)
dictionary_negative=pd.read_csv("sent_negative.csv",header=None)
dictionary_litigious=pd.read_csv("sent_litigious.csv",header=None)
dictionary_strongmodal=pd.read_csv("sent_strongmodal.csv",header=None)
dictionary_constraining=pd.read_csv("sent_constraining.csv",header=None)
dictionary_weakmodal=pd.read_csv("sent_weakmodal.csv",header=None)
dictionary_uncertainity=pd.read_csv("sent_uncertainity.csv",header=None)
dictionary_master=pd.read_csv("master.csv")

In [11]:
#Load the csv news feed file into pandas dataframe(make changes here afterwards)
data=pd.read_csv("news_feed1.csv")

In [44]:
def news_data_preprocessing(data):
    print("Number of NA/missing date values:")
    print(len(data)-data['text'].count())
    cleaned_data=data.dropna(subset=["published_date","title","description","text"])
    print("Number of NA/missing date values:")
    print(len(cleaned_data)-cleaned_data['text'].count())
    return cleaned_data

In [13]:
def sort_by_date(data):
    data['published_date'] =pd.to_datetime(data.published_date)
    data.sort_values(by=['published_date'], inplace=True, ascending=False)
    sorted_by_date_data=data.set_index('published_date')
    return sorted_by_date_data

In [14]:
#Apply Vader Sentiment Analysis on filtered words after matching with master dictionary
def custom_NLP1(text):
    pos_words=custom_word_selection(text)
    matched_words=match_words_master(dictionary_master,pos_words)
    value=normalize_scores(matched_words)
    return value

In [15]:
#Apply Vader Sentiment Analysis on featured words after matching with master dictionary
def custom_NLP2(text):
    pos_words=custom_word_selection(text)
    featured_pos_words=custom_feature_selection(pos_words)
    matched_words=match_words_master(dictionary_master,pos_words)
    value=normalize_scores(matched_words)
    return value

In [16]:
#Apply TextBlob Sentiment Analysis on filtered words after matching with master dictionary
def custom_NLP3(text):
    pos_words=custom_word_selection(text)
    matched_words=match_words_master(dictionary_master,pos_words)
    value=normalize_scores1(matched_words)
    return value

In [17]:
#Apply TextBlob Sentiment Analysis on featured words after matching with master dictionary
def custom_NLP4(text):
    pos_words=custom_word_selection(text)
    featured_pos_words=custom_feature_selection(pos_words)
    matched_words=match_words_master(dictionary_master,pos_words)
    value=normalize_scores1(matched_words)
    return value

In [18]:
#Apply Vader Sentiment Analysis on filtered words without matching with master dictionary
def custom_NLP5(text):
    pos_words=custom_word_selection(text)
    value=normalize_scores(pos_words[0])
    return value

In [19]:
#Apply Vader Sentiment Analysis on featured words without matching with master dictionary
def custom_NLP6(text):
    pos_words=custom_word_selection(text)
    featured_pos_words=custom_feature_selection(pos_words)
    value=normalize_scores(featured_pos_words[0])
    return value

In [20]:
#Apply Vader Sentiment Analysis on filtered words without matching with master dictionary
def custom_NLP7(text):
    pos_words=custom_word_selection(text)
    value=normalize_scores1(pos_words[0])
    return value

In [21]:
#Apply Vader Sentiment Analysis on featured words without matching with master dictionary
def custom_NLP8(text):
    pos_words=custom_word_selection(text)
    featured_pos_words=custom_feature_selection(pos_words)
    value=normalize_scores1(featured_pos_words[0])
    return value

In [22]:
data["Title_textblob"]=data["title"].apply(predict_sentiment_textblob)

In [23]:
#Applying these functions on the text and description columns and store them as columns in new dataframe
# def apply_functions(data):
#     data["Title_textblob"]=data["title"].apply(predict_sentiment_textblob)
#     data["Text_textblob"]=data["text"].apply(predict_sentiment_textblob)
#     data["Description_textblob"]=data["description"].apply(predict_sentiment_textblob)
#     data["Title_Vader"]=data["title"].apply(predict_sentiment_vader)
#     data["Text_Vader"]=data["text"].apply(predict_sentiment_vader)
#     data["Description_Vader"]=data["description"].apply(predict_sentiment_vader)
#     data["Custom_NLP1_ontitle"]=data["title"].apply(custom_NLP1)
#     data["Custom_NLP1_ontext"]=data["text"].apply(custom_NLP1)
#     data["Custom_NLP1_ondescription"]=data["description"].apply(custom_NLP1)
#     data["Custom_NLP2_ontitle"]=data["title"].apply(custom_NLP2)
#     data["Custom_NLP2_ontext"]=data["text"].apply(custom_NLP2)
#     data["Custom_NLP2_ondescription"]=data["description"].apply(custom_NLP2)
#     data["Custom_NLP3_ontitle"]=data["title"].apply(custom_NLP3)
#     data["Custom_NLP3_ontext"]=data["text"].apply(custom_NLP3)
#     data["Custom_NLP3_ondescription"]=data["description"].apply(custom_NLP3)
#     data["Custom_NLP4_ontitle"]=data["title"].apply(custom_NLP4)
#     data["Custom_NLP4_ontext"]=data["text"].apply(custom_NLP4)
#     data["Custom_NLP4_ondescription"]=data["description"].apply(custom_NLP4)
#     data["Custom_NLP5_ontitle"]=data["title"].apply(custom_NLP5)
#     data["Custom_NLP5_ontext"]=data["text"].apply(custom_NLP5)
#     data["Custom_NLP5_ondescription"]=data["description"].apply(custom_NLP5)
#     data["Custom_NLP6_ontitle"]=data["title"].apply(custom_NLP6)
#     data["Custom_NLP6_ontext"]=data["text"].apply(custom_NLP6)
#     data["Custom_NLP6_ondescription"]=data["description"].apply(custom_NLP6)
#     data["Custom_NLP7_ontitle"]=data["title"].apply(custom_NLP7)
#     data["Custom_NLP7_ontext"]=data["text"].apply(custom_NLP7)
#     data["Custom_NLP7_ondescription"]=data["description"].apply(custom_NLP7)
#     data["Custom_NLP8_ontitle"]=data["title"].apply(custom_NLP8)
#     data["Custom_NLP8_ontext"]=data["text"].apply(custom_NLP8)
#     data["Custom_NLP8_ondescription"]=data["description"].apply(custom_NLP8)

In [24]:
data["Text_textblob"]=data["text"].apply(predict_sentiment_textblob)

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>

In [25]:
data

,Unnamed: 0,_id,description,published_date,source,text,title,Title_textblob
0,0,5c8fb128fed8a82350ec58ec,Hotel Leela Venture is selling of some of its ...,2019-03-19,http://www.bloombergquint.com/business/hotel-l...,Hotel Leela said the proceeds from the slump s...,"Hotel Leelaventure To Sell Four Hotels, Agra P...",0.000000
1,1,5c8fb128fed8a82350ec58ed,Racing against time to meet the Supreme Court’...,2019-03-18,http://www.bloombergquint.com/business/relianc...,A source in the know said RCom has paid Rs 458...,Reliance Communications Pays Rs 458.77 Crore T...,0.000000
2,2,5c8fb128fed8a82350ec58ee,Lyft Inc. is seeking to raise as much as $2.1 ...,2019-03-19,http://www.bloombergquint.com/business/lyft-ai...,The filing with the U.S. Securities & Exchange...,Lyft Aims for Valuation Near $20 Billion in Bi...,0.100000
3,3,5c8fb128fed8a82350ec58ef,The appellate tribunal refused to stay the Nat...,2019-03-18,http://www.bloombergquint.com/law-and-policy/n...,The appellate tribunal refused to stay the Nat...,NCLAT Denies Stay On ArcelorMittal’s Takeover ...,0.000000
4,4,5c8fb128fed8a82350ec58f0,Mindtree Ltd. needs a new white knight to fend...,2019-03-18,http://www.bloombergquint.com/business/mindtre...,Mindtree Ltd. needs a new white knight to fend...,Mindtree Needs A White Knight To Replace Siddh...,0.000000
5,5,5c8fb128fed8a82350ec58f1,Revenue climbed 36 percent in January and Febr...,2019-03-18,http://www.bloombergquint.com/business/huawei-...,After years of tension and accusations that th...,Huawei Defies Global Troubles With Acceleratin...,0.000000
6,6,5c8fb128fed8a82350ec58f2,The auto major produced a total of 1.49 lakh u...,2019-03-18,http://www.bloombergquint.com/business/maruti-...,The second production line has been commission...,Maruti Cuts Vehicle Production By Over 8% In F...,0.000000
7,7,5c8fb128fed8a82350ec58f3,The Federal Reserve will bring the current cyc...,2019-03-19,http://www.bloombergquint.com/global-economics...,The median of responses in the March 13-15 pol...,Fed Expected to Hit Rate Cycle’s Peak After On...,0.200000
8,8,5c8fb128fed8a82350ec58f4,Fidelity National Information Services Inc. ag...,2019-03-19,http://www.bloombergquint.com/business/fis-to-...,Payments companies earn fees from charging to ...,FIS’s $34 Billion Bid for Worldpay Drives Paym...,0.000000
9,9,5c8fb128fed8a82350ec58f5,Chinese leader Xi Jinping will make state visi...,2019-03-18,http://www.bloombergquint.com/business/xi-to-v...,There has been speculation that Xi and U.S. Pr...,Xi Jinping To Visit Europe In Bid To Boost Trade,0.000000


In [23]:
def main(data):
    cleaned_data=news_data_preprocessing(data)
    sorted_by_date_data=sort_by_date(cleaned_data)
    sorted_by_date_data["Title_textblob"]=sorted_by_date_data["title"].apply(predict_sentiment_textblob)
    sorted_by_date_data["Text_textblob"]=sorted_by_date_data["text"].apply(predict_sentiment_textblob)
    sorted_by_date_data["Description_textblob"]=sorted_by_date_data["description"].apply(predict_sentiment_textblob)
    sorted_by_date_data["Title_vader"]=sorted_by_date_data["title"].apply(predict_sentiment_vader)
    sorted_by_date_data["Text_vader"]=sorted_by_date_data["text"].apply(predict_sentiment_vader)
    sorted_by_date_data["Description_vader"]=sorted_by_date_data["description"].apply(predict_sentiment_vader)
    return sorted_by_date_data

In [25]:
sorted_by_date_data=main(data)

Number of NA/missing date values:
168
Number of NA/missing date values:
0


E:\Software\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Software\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
sorted_by_date_data

,Unnamed: 0,_id,description,source,text,title,Title_textblob,Text_textblob,Description_textblob,Title_vader,Text_vader,Description_vader
published_date,,,,,,,,,,,,
2019-04-24,8390,5cc039223120522d583a92ce,(Bloomberg) -- Verizon Communications Inc. pos...,https://www.bloombergquint.com/technology/veri...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Verizon Raises Outlook But Wireless Subscriber...,0.000000,0.094286,0.433333,"(0.282, 0.173, 0.296, 0.545)","(0.07, 0.0, 0.5574, 0.93)","(0.204, 0.0, 0.8271, 0.796)"
2019-04-24,7992,5cc0051e31205227f8149dd4,(Bloomberg Opinion) -- The S&P 500 Index set a...,https://www.bloombergquint.com/global-economic...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Stock Bulls Have Come to a Fork in the Road,0.000000,0.094286,0.026250,"(0.0, 0.0, 0.0, 1.0)","(0.07, 0.0, 0.5574, 0.93)","(0.046, 0.0, 0.2716, 0.954)"
2019-04-24,7872,5cc0051e31205227f8149d53,More than a third of the debt NBFCs raised fro...,http://www.bloombergquint.com/business/this-is...,More than a third of the debt non-bank lenders...,This Is How Much NBFCs Have To Repay Mutual Fu...,0.200000,0.125000,0.166667,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.075, -0.3612, 0.925)","(0.0, 0.116, -0.3612, 0.884)"
2019-04-24,7871,5cc0051e31205227f8149d52,Lupin Ltd. said today one of its units in Pith...,http://www.bloombergquint.com/markets/lupin-fa...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Lupin Falls After U.S. FDA Classifies Pithampu...,0.100000,0.094286,0.100000,"(0.0, 0.0, 0.0, 1.0)","(0.07, 0.0, 0.5574, 0.93)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,7870,5cc0051e31205227f8149d51,"Investment and consumption waned in India, lat...",http://www.bloombergquint.com/global-economics...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Slackening Consumer Demand Reins In Animal Spi...,0.000000,0.094286,0.281250,"(0.0, 0.2, -0.128, 0.8)","(0.07, 0.0, 0.5574, 0.93)","(0.12, 0.0, 0.3818, 0.88)"
2019-04-24,7868,5cc0051e31205227f8149d4f,Digital gaming startup Mobile Premier League r...,http://www.bloombergquint.com/technology/this-...,Bloomberg\nBloomberg | Quint is a multiplatfor...,This Seven-Month-Old Gaming Startup Tops Up $3...,0.000000,0.094286,-0.400000,"(0.32, 0.0, 0.5106, 0.68)","(0.07, 0.0, 0.5574, 0.93)","(0.103, 0.0, 0.3182, 0.897)"
2019-04-24,6641,5cbfe5e531205227f814987c,"Get latest Stock Market Advice, Stock Market I...",http://www.moneycontrol.com/news/investing-298...,"investing\nApril 23, 2019 09:03 PM IST Gold fa...","Stock Market Advice, Stock Market Investment A...",0.000000,0.076719,0.300000,"(0.0, 0.0, 0.0, 1.0)","(0.107, 0.055, 0.9948, 0.838)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,6650,5cbfe5e531205227f8149885,"Get Tax, Tax News, Latest Tax Updates, Tax Ind...",http://www.moneycontrol.com/news/tax-292.html,"tax\nApril 18, 2019 05:43 PM IST Viewpoint | H...","Tax, Tax News, Latest Tax Updates, Tax India",0.500000,0.059277,0.300000,"(0.0, 0.0, 0.0, 1.0)","(0.045, 0.039, -0.2237, 0.917)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,6658,5cbfe5e531205227f814988d,Get all latest & breaking news on Real Estate....,http://www.moneycontrol.com/news/tags/real-est...,"real estate\nApril 23, 2019 06:44 PM IST Embas...","Photos, Videos, Breaking Stories and Articles ...",0.200000,0.142031,0.350000,"(0.0, 0.0, 0.0, 1.0)","(0.039, 0.033, 0.3653, 0.928)","(0.091, 0.0, 0.2023, 0.909)"


In [48]:
def main1(data):
    sorted_by_date_data=news_data_preprocessing1(data)
    sorted_by_date_data["Title_custom_NLP1"]=sorted_by_date_data["title"].apply(custom_NLP1)
    return sorted_by_date_data

In [49]:
sorted_by_date_data1=main1(sorted_by_date_data)

Number of NA/missing date values:
0
Number of NA/missing date values:
0


In [47]:
def news_data_preprocessing1(data):
    print("Number of NA/missing date values:")
    print(len(data)-data['text'].count())
    cleaned_data=data.dropna(subset=["title","description","text"])
    print("Number of NA/missing date values:")
    print(len(cleaned_data)-cleaned_data['text'].count())
    return cleaned_data

In [50]:
sorted_by_date_data1

,Unnamed: 0,_id,description,source,text,title,Title_textblob,Text_textblob,Description_textblob,Title_vader,Text_vader,Description_vader,Title_custom_NLP1
published_date,,,,,,,,,,,,,
2019-04-24,8390,5cc039223120522d583a92ce,(Bloomberg) -- Verizon Communications Inc. pos...,https://www.bloombergquint.com/technology/veri...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Verizon Raises Outlook But Wireless Subscriber...,0.000000,0.094286,0.433333,"(0.282, 0.173, 0.296, 0.545)","(0.07, 0.0, 0.5574, 0.93)","(0.204, 0.0, 0.8271, 0.796)","(0.14285714285714285, 0.14285714285714285, 0.0..."
2019-04-24,7992,5cc0051e31205227f8149dd4,(Bloomberg Opinion) -- The S&P 500 Index set a...,https://www.bloombergquint.com/global-economic...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Stock Bulls Have Come to a Fork in the Road,0.000000,0.094286,0.026250,"(0.0, 0.0, 0.0, 1.0)","(0.07, 0.0, 0.5574, 0.93)","(0.046, 0.0, 0.2716, 0.954)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,7872,5cc0051e31205227f8149d53,More than a third of the debt NBFCs raised fro...,http://www.bloombergquint.com/business/this-is...,More than a third of the debt non-bank lenders...,This Is How Much NBFCs Have To Repay Mutual Fu...,0.200000,0.125000,0.166667,"(0.0, 0.0, 0.0, 1.0)","(0.0, 0.075, -0.3612, 0.925)","(0.0, 0.116, -0.3612, 0.884)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,7871,5cc0051e31205227f8149d52,Lupin Ltd. said today one of its units in Pith...,http://www.bloombergquint.com/markets/lupin-fa...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Lupin Falls After U.S. FDA Classifies Pithampu...,0.100000,0.094286,0.100000,"(0.0, 0.0, 0.0, 1.0)","(0.07, 0.0, 0.5574, 0.93)","(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,7870,5cc0051e31205227f8149d51,"Investment and consumption waned in India, lat...",http://www.bloombergquint.com/global-economics...,Bloomberg\nBloomberg | Quint is a multiplatfor...,Slackening Consumer Demand Reins In Animal Spi...,0.000000,0.094286,0.281250,"(0.0, 0.2, -0.128, 0.8)","(0.07, 0.0, 0.5574, 0.93)","(0.12, 0.0, 0.3818, 0.88)","(0.0, 0.14285714285714285, -0.0182857142857142..."
2019-04-24,7868,5cc0051e31205227f8149d4f,Digital gaming startup Mobile Premier League r...,http://www.bloombergquint.com/technology/this-...,Bloomberg\nBloomberg | Quint is a multiplatfor...,This Seven-Month-Old Gaming Startup Tops Up $3...,0.000000,0.094286,-0.400000,"(0.32, 0.0, 0.5106, 0.68)","(0.07, 0.0, 0.5574, 0.93)","(0.103, 0.0, 0.3182, 0.897)","(0.16666666666666666, 0.0, 0.08510000000000001..."
2019-04-24,6641,5cbfe5e531205227f814987c,"Get latest Stock Market Advice, Stock Market I...",http://www.moneycontrol.com/news/investing-298...,"investing\nApril 23, 2019 09:03 PM IST Gold fa...","Stock Market Advice, Stock Market Investment A...",0.000000,0.076719,0.300000,"(0.0, 0.0, 0.0, 1.0)","(0.107, 0.055, 0.9948, 0.838)","(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,6650,5cbfe5e531205227f8149885,"Get Tax, Tax News, Latest Tax Updates, Tax Ind...",http://www.moneycontrol.com/news/tax-292.html,"tax\nApril 18, 2019 05:43 PM IST Viewpoint | H...","Tax, Tax News, Latest Tax Updates, Tax India",0.500000,0.059277,0.300000,"(0.0, 0.0, 0.0, 1.0)","(0.045, 0.039, -0.2237, 0.917)","(0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
2019-04-24,6658,5cbfe5e531205227f814988d,Get all latest & breaking news on Real Estate....,http://www.moneycontrol.com/news/tags/real-est...,"real estate\nApril 23, 2019 06:44 PM IST Embas...","Photos, Videos, Breaking Stories and Articles ...",0.200000,0.142031,0.350000,"(0.0, 0.0, 0.0, 1.0)","(0.039, 0.033, 0.3653, 0.928)","(0.091, 0.0, 0.2023, 0.909)","(0.0, 0.0, 0.0, 1.0)"
